In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10580,2024-05-16,Suécia Basketligan,14:04,Borås,Norrkoping,2.10,1.70,162.5,1.86,1.88,1.5,1.92,1.56,Q5AUF3Kr,0.476190,0.588235,0.537634,0.531915,0.064426,202.538,66.909753,0.330357,0.6,1.341641,2.236068,245.05,2.748,1.064716,0.387451,-31.0,106.274,19.230843,0.180955,3.0,0.000000,0.000000,82.50,1.206,0.032863,0.027250,96.0,65,66,3.77,1.25,112.930,121.890,0.148865,0.007563,0.146298,1.43,0.286,3.846154,0.799007,0.9,0.100993,0.55,0.110,6.363636,0.762785,0.8,0.037215
10581,2024-05-16,México Cibacopa,00:15,Caballeros de Culiacan,Rayos de Hermosillo,2.20,1.62,176.5,1.85,1.85,2.5,1.90,1.39,069xSuC6,0.454545,0.617284,0.540541,0.540541,0.071829,232.982,54.448237,0.233701,1.8,1.643168,0.912871,204.36,2.640,0.402927,0.152624,1.0,131.110,43.760698,0.333771,2.4,1.341641,0.559017,121.80,1.494,0.461552,0.308937,45.0,78,84,2.62,1.45,180.894,171.838,0.214724,0.000000,0.219225,-0.80,-0.160,-7.500000,0.468955,0.2,-0.268955,0.44,0.088,7.045455,0.503197,0.6,0.096803
10582,2024-05-16,México Cibacopa,00:15,Halcones de Obregon,Angeles CDMX,1.67,2.12,181.5,1.85,1.85,-3.5,2.03,2.91,rVbsRaRC,0.598802,0.471698,0.540541,0.540541,0.070501,163.642,22.436203,0.137105,1.8,1.643168,0.912871,162.24,1.726,0.271164,0.157106,24.0,216.810,87.770202,0.404825,0.6,1.341641,2.236068,193.44,2.300,0.689601,0.299827,-41.0,96,93,1.69,2.08,163.172,223.086,0.167915,0.000000,0.251925,2.74,0.548,1.222628,0.613340,0.8,0.186660,0.91,0.182,6.153846,0.536507,0.7,0.163493
10583,2024-05-16,México Cibacopa,00:15,Venados de Mazatlan,Pioneros de Los Mochis,1.51,2.45,175.5,1.85,1.85,0.0,0.00,0.00,f96UT1sf,0.662252,0.408163,0.540541,0.540541,0.070415,169.806,49.465373,0.291305,1.2,1.643168,1.369306,148.52,2.156,0.851487,0.394938,-14.0,172.096,54.944933,0.319269,0.6,1.341641,2.236068,199.23,1.878,0.548516,0.292075,-27.0,94,87,1.58,2.29,167.952,276.598,0.335697,0.000000,NaN,2.39,0.478,1.066946,0.595560,0.7,0.104440,-5.00,-1.000,-1.450000,0.409908,0.2,-0.209908
10584,2024-05-16,Nova Zelândia Nbl,04:30,Otago Nuggets,Auckland Tuatara,4.42,1.18,185.5,1.85,1.85,0.0,0.00,0.00,pEuoOPO0,0.226244,0.847458,0.540541,0.540541,0.073702,193.676,97.322041,0.502499,0.6,1.341641,2.236068,219.45,2.240,1.154924,0.515591,-44.0,129.344,41.446979,0.320440,3.0,0.000000,0.000000,113.30,1.204,0.326466,0.271151,133.0,105,110,2.09,1.03,0.000,141.240,0.818224,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-0.48,-0.096,-1.875000,0.000000,0.0,0.000000
10585,2024-05-16,República Tcheca Nbl,13:00,Usti n. Labem,Decin,1.46,2.59,161.5,1.80,1.86,0.0,0.00,0.00,GthuSaj2,0.684932,0.386100,0.555556,0.537634,0.071032,141.044,37.477262,0.265713,1.8,1.643168,0.912871,122.12,1.744,0.518151,0.297105,-4.0,197.402,67.425871,0.341566,1.2,1.643168,1.369306,204.00,2.408,0.787921,0.327210,4.0,86,75,1.42,2.72,108.668,219.456,0.394583,0.023184,NaN,0.24,0.048,9.583333,0.718492,0.9,0.181508,2.71,0.542,2.933579,0.440534,0.5,0.059466
10586,2024-05-16,Rússia Liga Vtb United,13:30,Unics Kazan,Lokomotiv Kuban,1.55,2.47,158.5,1.87,1.92,0.0,0.00,0.00,UNWHaM1C,0.645161,0.404858,0.534759,0.520833,0.050020,144.086,45.248695,0.314039,1.8,1.643168,0.912871,204.00,1.760,0.487186,0.276810,24.0,166.782,57.415766,0.344256,1.8,1.643168,0.912871,107.44,2.092,0.683206,0.326580,27.0,85,68,2.40,1.58,107.926,179.262,0.323651,0.018657,NaN,0.35,0.070,7.857143

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10582,00:15,México Cibacopa,Halcones de Obregon,Angeles CDMX,1.67,Back Home
10598,21:00,Porto Rico Bsn,San German,Gigantes de Carolina,1.56,Back Home
10613,21:30,Argentina Liga A,La Union,San Lorenzo,1.71,Back Home
